# 0. 준비

In [ ]:
"""_summary_
* create environment for essentia(to get key signature)
* it is hard to use on Colab or Windows, so you can try this on mac
"""

# conda create -n key python=3.9
# conda activate key
# brew install fftw libyaml libsamplerate eigen taglib chromaprint ffmpeg
# pip install "numpy<2.0"
# pip install essentia
# pip install pandas tqdm

'_summary_\n* create environment for essentia(to get key signature)\n* it is hard to use on Colab or Windows, so you can try this on mac\n'

In [4]:
from pathlib import Path

In [5]:
import essentia
import essentia.standard as es

# 1. 조성 분포

In [7]:
def extract_key_signature(audio_path):
    loader = es.MonoLoader(filename=str(audio_path))
    audio = loader()
    
    key_extractor = es.KeyExtractor()
    key, scale, strength = key_extractor(audio)
    return key, scale, strength

---

In [3]:
import pandas as pd

df = pd.read_csv('eda_full.csv', encoding='utf-8-sig')
df.head()

,video,yt_id,duration,title,singer,audio,lyric,year,match,pn,tempo_transition_complexity,active_bpm_variance,pitch_high,pitch_low,pitch_range,rms_variation,repeat_area_size,repetition_ratio
0,아이유(IU) - Blueming(블루밍) [가사_Lyrics],yCFMJrneOUs,3:38,Blueming,IU,Q1,Q1,2019,2,0,0.073658,49.098753,618.670048,65.406391,553.263657,0.015928,52239650,0.602505
1,아이유(IU) - 좋은 날 [가사_Lyrics],V6WWJNpIJN4,3:56,좋은날,IU,Q1,Q1,2010,2,0,0.194658,801.429437,410.534516,65.406391,345.128125,0.012732,74291112,0.727768
2,아이유(IU) - 이 지금 [가사_Lyrics],RUuRcR7ZQUg,2:57,이 지금,IU,Q4,Q1,2017,2,0,0.193476,800.170063,880.000000,65.406391,814.593609,0.005314,26916160,0.470882
3,다비치 - 안녕이라고 말하지마 [가사_Lyrics],uw83-MnCpAo,3:51,안녕이라고 말하지마,다비치,Q4,Q4,2011,2,0,0.109879,1051.305489,959.646805,65.406391,894.240413,0.003815,57633998,0.590724
4,다비치 - 거북이 [가사_Lyrics],ofBinMrHfHQ,3:44,거북이,다비치,Q1,Q1,2013,2,0,0.178801,455.823726,177.666317,65.406391,112.259925,0.005987,50365320,0.546556


In [9]:
df['key'] = ''
df['scale'] = 0
df['strength'] = 0

In [10]:
pos = Path('downloads/2')
pos_wav_paths = sorted(pos.glob('*.wav'))
neg = Path('downloads/3')
neg_wav_paths = sorted(neg.glob('*.wav'))

In [12]:
from tqdm.auto import tqdm
for wav_path in tqdm(pos_wav_paths):
    video_name = wav_path.stem
    key, scale, strength = extract_key_signature(wav_path)
    df.loc[df['video'] == video_name, 'key'] = key
    df.loc[df['video'] == video_name, 'scale'] = scale
    df.loc[df['video'] == video_name, 'strength'] = strength
for wav_path in tqdm(neg_wav_paths):
    video_name = wav_path.stem
    key, scale, strength = extract_key_signature(wav_path)
    df.loc[df['video'] == video_name, 'key'] = key
    df.loc[df['video'] == video_name, 'scale'] = scale
    df.loc[df['video'] == video_name, 'strength'] = strength

/opt/anaconda3/envs/ess/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/30 [00:00<?, ?it/s]/var/folders/8s/9mw0byfj2yjdqnsnrkq34mkc0000gn/T/ipykernel_1099/1079498059.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'minor' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['video'] == video_name, 'scale'] = scale
/var/folders/8s/9mw0byfj2yjdqnsnrkq34mkc0000gn/T/ipykernel_1099/1079498059.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8332850933074951' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['video'] == video_n

In [14]:
df.iloc[0]

video                          아이유(IU) - Blueming(블루밍) [가사_Lyrics]
yt_id                                                  yCFMJrneOUs
duration                                                      3:38
title                                                     Blueming
singer                                                          IU
audio                                                           Q1
lyric                                                           Q1
year                                                          2019
match                                                            2
pn                                                               0
tempo_transition_complexity                               0.073658
active_bpm_variance                                      49.098753
pitch_high                                              618.670048
pitch_low                                                65.406391
pitch_range                                             553.26

In [15]:
df.to_csv('eda_full.csv', encoding='utf-8-sig', index=False)